In [2]:
import models
import utils
import torch
import torch.nn as nn
import torch.nn.functional as f
import numpy as np
import matplotlib.pyplot as plt
from torch import optim
from torch.autograd import Variable


%load_ext autoreload
%autoreload 2

torch.manual_seed(42)

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

Y = torch.tensor(np.load('../datasets/floored_exp_uncorrel.npy'), dtype=dtype)
A_true = np.load('../datasets/students_uncorrel.npy')
D_true = np.load('../datasets/questions_uncorrel.npy')
# We assume we know the relevant concept of each question beforehand
concepts = np.nonzero(D_true)
num_students, num_concepts = A_true.shape
num_questions = D_true.shape[0]
guess_prob = 1/5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def accuracy(R_pred, R_true):
    R_pred = R_pred.data.numpy()
    R_true = R_true.data.numpy()
    R_pred_cpy = np.copy(R_pred)
    R_pred_cpy[R_pred_cpy > 0.5] = 1
    R_pred_cpy[R_pred_cpy <= 0.5] = 0
    print("Accuracy: {}".format(np.sum(R_pred_cpy == R_true) / (R_true.shape[0]*R_true.shape[1])))
    return np.sum(R_pred_cpy == R_true) / (R_true.shape[0]*R_true.shape[1])

In [4]:
def rmse_min_max(A,B):
    a_norm = (A - A.min())/(A.max()-A.min())
    b_norm = (B - B.min())/(B.max()-B.min())
    return np.sqrt(np.mean(np.square(a_norm-b_norm))) 

In [5]:
Yt = Y.transpose(0,1)

## Training Simple RNN to predict students performance

In [22]:
n_epochs = 3000
hidden_size = 128

model = models.RNN_Model(hidden_size, num_layers=8)
criterion = nn.functional.binary_cross_entropy
optimizer = optim.Adam(model.parameters(), lr=0.002, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [23]:
losses = np.zeros(n_epochs) # For plotting

for epoch in range(n_epochs):

    inputs = Variable(Yt[:-1])
    targets = Variable(Yt[1:])

    outputs, hidden = model(inputs, None)

    optimizer.zero_grad()
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    losses[epoch] += loss.data[0]

    if epoch % 20 == 0:
        print(epoch, loss.data[0])
        accuracy(outputs,targets)

/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app
/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 tensor(0.6939)
Accuracy: 0.49714141414141416
20 tensor(0.6272)
Accuracy: 0.6494545454545455
40 tensor(0.6243)
Accuracy: 0.6517575757575758
60 tensor(0.6240)
Accuracy: 0.6521111111111111
80 tensor(0.6232)
Accuracy: 0.6525252525252525
100 tensor(0.6212)
Accuracy: 0.6534545454545454
120 tensor(0.6178)
Accuracy: 0.6548282828282829
140 tensor(0.6125)
Accuracy: 0.659060606060606
160 tensor(0.6068)
Accuracy: 0.6636969696969697
180 tensor(0.6009)
Accuracy: 0.6687070707070707
200 tensor(0.5945)
Accuracy: 0.6750101010101011
220 tensor(0.5879)
Accuracy: 0.6809292929292929
240 tensor(0.5821)
Accuracy: 0.6856464646464646
260 tensor(0.5776)
Accuracy: 0.688949494949495
280 tensor(0.5733)
Accuracy: 0.691949494949495
300 tensor(0.5700)
Accuracy: 0.6933838383838384
320 tensor(0.5651)
Accuracy: 0.6976969696969697
340 tensor(0.5619)
Accuracy: 0.7003333333333334
360 tensor(0.5573)
Accuracy: 0.702929292929293
380 tensor(0.5530)
Accuracy: 0.705060606060606
400 tensor(0.5474)
Accuracy: 0.7094747474747475
42

KeyboardInterrupt: 

## Training Skill RNN to predict students performance

In [6]:
n_epochs = 3000
hidden_size = 128
dropout = 0.2
num_layers = 8
average = True
sigmoid = False

concepts = np.nonzero(D_true)
model = models.RNN_Skills_Model(average, concepts, num_concepts, num_questions, hidden_size, num_students, num_layers, dropout, sigmoid)
criterion = nn.functional.binary_cross_entropy
optimizer = optim.Adam(model.parameters(), lr=0.002, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [8]:
losses = np.zeros(n_epochs) # For plotting

for epoch in range(n_epochs):

    inputs = Variable(Yt[:-1])
    targets = Variable(Yt[1:])

    outputs, hidden, skills, D = model(inputs, None)

    optimizer.zero_grad()
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    losses[epoch] += loss.data[0]

    if epoch % 20 == 0:
        print(epoch, loss.data[0])
        accuracy(outputs,targets)
        print('RMSE A: {}'.format(rmse_min_max(skills.data.numpy(), A_true)))
        print('RMSE D: {}'.format(rmse_min_max(D.data.numpy(), D_true)))

/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  from ipykernel import kernelapp as app
/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 tensor(0.8247)
Accuracy: 0.5113333333333333
RMSE A: 0.30150626692352633
RMSE D: 0.3110942077766168
20 tensor(0.8247)
Accuracy: 0.5113333333333333
RMSE A: 0.30279388112519023
RMSE D: 0.3110942077766168
40 tensor(0.8247)
Accuracy: 0.5113333333333333
RMSE A: 0.30141495837589816
RMSE D: 0.3110942077766168
60 tensor(0.8247)
Accuracy: 0.5113333333333333
RMSE A: 0.30250240916832477
RMSE D: 0.3110942077766168
80 tensor(0.8247)
Accuracy: 0.5113333333333333
RMSE A: 0.3008059640369871
RMSE D: 0.3110942077766168


KeyboardInterrupt: 

In [32]:
#Hyperparameter Search
n_epochs = 50
hidden_size = {512,256,128,64}
dropout = np.random.uniform(size=5)
print(dropout)
num_layers = {32,16,8,4}
#l_rate = np.random.uniform(0.01,0.5,5)
l_rate = {0.002,0.01,0.05,0.2,0.5}
print(l_rate)
A_rmse = 1
D_rmse = 1
max_acc = 0
params = {}
concepts = np.nonzero(D_true)
acc = []
h_loss = {}
i=0
for size in hidden_size:
    for drop in dropout:
        for layers in num_layers:
            for rate in l_rate:
                model = models.RNN_Skills_Model(False, concepts, num_concepts, num_questions, size, num_students, layers, drop)
                criterion = nn.functional.binary_cross_entropy
                optimizer = optim.Adam(model.parameters(), lr=rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

                losses = np.zeros(n_epochs)
                
                for epoch in range(n_epochs):

                    inputs = Variable(Yt[:-1])
                    targets = Variable(Yt[1:])

                    outputs, hidden, skills, D = model(inputs, None)

                    optimizer.zero_grad()
                    loss = criterion(outputs, targets)
                    loss.backward()
                    optimizer.step()
                    
                    losses[epoch] += loss.data[0]
                    
                    if epoch>0 and np.abs(losses[epoch]-losses[epoch-1])<0.00005:
                        break

                    if epoch % 10 == 0:
                        print(epoch, loss.data[0])
                        print(accuracy(outputs,targets))
                        print('RMSE A: {}'.format(rmse_min_max(skills.data.numpy(), A_true)))
                        print('RMSE D: {}'.format(rmse_min_max(D.data.numpy(), D_true)))

                
                print(accuracy(outputs,targets))
                if accuracy(outputs,targets)>max_acc:
                    params['accuracy'] = (size,drop,layers,rate)
                    max_acc = accuracy(outputs,targets)
                if rmse_min_max(skills.data.numpy(), A_true)<A_rmse:
                    params['A'] = (size,drop,layers,rate)
                    A_rmse = rmse_min_max(skills.data.numpy(), A_true)
                if rmse_min_max(D.data.numpy(), D_true)<D_rmse:
                    params['D'] = (size,drop,layers,rate)
                    D_rmse = rmse_min_max(D.data.numpy(), D_true)
                acc.append(accuracy(outputs,targets))
                h_loss[i] = losses
                print('done')
                i+=1
                    
print(params)
print(max_acc)
print(A_rmse)
print(D_rmse)
print(acc)

[0.67363352 0.34875009 0.45567222 0.76393018 0.62340218]
{0.5, 0.1, 0.3, 0.2, 0.02}


/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 tensor(1.3280)
0.48944444444444446
RMSE A: 0.41321895522657626
RMSE D: 0.46254148619455304
10 tensor(0.8601)
0.4803131313131313
RMSE A: 0.36741722820092293
RMSE D: 0.3854353724191575
0.4787070707070707
done
0 tensor(1.9985)
0.5032929292929293
RMSE A: 0.50919118529673
RMSE D: 0.4492963114053607
10 tensor(0.8606)
0.4785656565656566
RMSE A: 0.36756588297951304
RMSE D: 0.3713725979540209
20 tensor(0.8554)
0.48397979797979795
RMSE A: 0.3641292753668714
RMSE D: 0.42039191691995553
0.48597979797979796
done
0 tensor(1.2593)
0.5110707070707071
RMSE A: 0.48182128081911146
RMSE D: 0.330943770978714
0.4784242424242424
done
0 tensor(2.4409)
0.5184444444444445
RMSE A: 0.47526985063165056
RMSE D: 0.34508595146288723
10 tensor(0.8406)
0.5003636363636363
RMSE A: 0.35819794728253196
RMSE D: 0.43959553735739854
20 tensor(0.8348)
0.5008181818181818
RMSE A: 0.3194916266610254
RMSE D: 0.4446673889964636
0.5054848484848485
done
0 tensor(1.4675)
0.4865959595959596
RMSE A: 0.4976868465537525
RMSE D: 0.387302

KeyboardInterrupt: 